In [47]:
import pandas as pd
import plotly.express as px

def carregar_dades(ciutat):
    # Població
    df_poblacio = pd.read_csv(f'dat/{ciutat}/poblacio.csv', skiprows=10, sep=";", decimal=",")
    df_poblacio.rename(columns={"Unnamed: 0":"Any", "Total":"Població"}, inplace=True)
    df_poblacio = df_poblacio.set_index("Any")
    df_poblacio = df_poblacio[['Població']]
    
    # Densitat
    df_densitat = pd.read_csv(f'dat/{ciutat}/densitat.csv', skiprows=6, sep=";", decimal=',')
    df_densitat.rename(columns={"Unnamed: 0":"Any", "Superfície (km²)":"Superfície", "Densitat (hab./km²)":"Densitat"}, inplace=True)
    df_densitat.set_index("Any", inplace=True)
    df_densitat = df_densitat[['Superfície', 'Densitat']]
    
    # Naixements
    df_naixements = pd.read_csv(f'dat/{ciutat}/naixements.csv', skiprows=8, sep=";")
    df_naixements.rename(columns={"Unnamed: 0":"Any", "Total":"Naixements"}, inplace=True)
    df_naixements.set_index("Any", inplace=True)
    df_naixements = df_naixements[['Naixements']]
    
    # Defuncions
    df_defuncions = pd.read_csv(f'dat/{ciutat}/defuncions.csv', skiprows=7, sep=";")
    df_defuncions.rename(columns={"Unnamed: 0":"Any", "Total":"Defuncions"}, inplace=True)
    df_defuncions.set_index("Any", inplace=True)
    df_defuncions = df_defuncions[['Defuncions']]

    # Aturats
    df_atur = pd.read_csv(f'dat/{ciutat}/atur.csv', skiprows=6, sep=";", decimal=',')
    df_atur.rename(columns={"Unnamed: 0":"Any", "Sexe. Total":"Aturats"}, inplace=True)
    df_atur = df_atur.set_index("Any")
    df_atur = df_atur[['Aturats']]

    # Pensionistes
    df_pensionistes = pd.read_csv(f'dat/{ciutat}/pensionistes.csv', skiprows=7, sep=";")
    df_pensionistes.rename(columns={'Unnamed: 0':'Any', 'Total':'Pensionistes'}, inplace=True)
    df_pensionistes = df_pensionistes.set_index("Any")
    df_pensionistes = df_pensionistes[['Pensionistes']]

    # Habitatges acabats
    df_habit = pd.read_csv(f'dat/{ciutat}/habitatges_construits.csv', skiprows=10, sep=";")
    noves_col = {
        'Unnamed: 0':'Any',
        'Habitatges iniciats amb protecció oficial':'HPO iniciats',
        'Habitatges acabats amb protecció oficial':'HPO acabats'
    }
    df_habit.rename(columns=noves_col, inplace=True)
    df_habit.set_index('Any', inplace=True)
    df_habit = df_habit[['HPO acabats', 'Habitatges acabats']]

    # Vehicles
    df_vehicles = pd.read_csv(f'dat/{ciutat}/vehicles.csv', skiprows=6, sep=";")
    df_vehicles.rename(columns={'Unnamed: 0':'Any'}, inplace=True)
    df_vehicles.set_index('Any', inplace=True)
    df_vehicles = df_vehicles[['Turismes', 'Motocicletes']]

    # Renta Familiar Disponible per Habitant
    df_rdf = pd.read_csv(f'dat/{ciutat}/rfd.csv', skiprows=6, sep=";")
    df_rdf.rename(columns={"Unnamed: 0":"Any",'Per habitant (€)':'RFDH' }, inplace=True)
    df_rdf = df_rdf.set_index("Any")
    df_rdf = df_rdf[['RFDH']]
    
    df = pd.concat([df_poblacio, df_densitat, df_naixements, df_defuncions,
                    df_atur, df_pensionistes, df_habit, df_vehicles, df_rdf], axis=1).sort_index(ascending=True)
    
    return df

def neteja_dades (df):
    df = df.drop(range(1975,2010))
    df.drop(2025, inplace=True)
    df['Població'] = df['Població'].fillna(round(df['Densitat']* df['Superfície'], 0))
    df.loc[2015, 'Pensionistes'] = round((df['Pensionistes'][2016]+df['Pensionistes'][2014])/2, 0)
    df.loc[2010, 'Pensionistes'] = df.loc[2011, 'Pensionistes']
    df.loc[2023, 'RFDH'] = df.loc[2022, 'RFDH']
    df.loc[2024, 'RFDH'] = df.loc[2022, 'RFDH']
    return df


In [59]:
ciutat = "Sant Cugat"
df_stcugat = carregar_dades (ciutat)
df_stcugat = neteja_dades (df_stcugat)
df_stcugat

,Població,Superfície,Densitat,Naixements,Defuncions,Aturats,Pensionistes,HPO acabats,Habitatges acabats,Turismes,Motocicletes,RFDH
Any,,,,,,,,,,,,
2010,81745.0,48.23,1694.9,996.0,381.0,3846.9,9133.0,72.0,174.0,35151.0,8095.0,23643.0
2011,84934.0,48.23,1727.9,1016.0,422.0,4035.0,9133.0,57.0,564.0,35712.0,8422.0,22676.0
2012,84947.0,48.23,1761.3,967.0,481.0,4285.6,9600.0,128.0,296.0,36074.0,8592.0,20811.0
2013,86110.0,48.23,1785.4,983.0,457.0,4285.5,10054.0,0.0,143.0,36332.0,8819.0,20828.0
2014,87118.0,48.23,1806.3,974.0,493.0,3936.9,10445.0,0.0,46.0,36760.0,9172.0,21079.0
2015,87832.0,48.23,1821.1,928.0,500.0,3505.4,10756.0,0.0,21.0,37612.0,9542.0,22719.0
2016,88922.0,48.23,1843.7,939.0,509.0,3116.0,11067.0,24.0,142.0,38379.0,10075.0,24014.0
2017,89515.0,48.23,1856.0,913.0,546.0,2723.3,11486.0,0.0,266.0,39374.0,10381.0,24391.0
2018,90663.0,48.23,1879.8,808.0,526.0,2574.3,11825.0,0.0,386.0,39955.0,10752.0,24835.0


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51 entries, 2025 to 1975
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Població            12 non-null     float64
 1   Superfície          28 non-null     float64
 2   Densitat            28 non-null     float64
 3   Naixements          50 non-null     float64
 4   Defuncions          50 non-null     float64
 5   Aturats             20 non-null     float64
 6   Pensionistes        13 non-null     float64
 7   HPO acabats         33 non-null     float64
 8   Habitatges acabats  33 non-null     float64
 9   Turismes            28 non-null     float64
 10  Motocicletes        28 non-null     float64
 11  RFDH                13 non-null     float64
dtypes: float64(12)
memory usage: 5.2 KB


In [26]:
df.isna().sum()

Població              39
Superfície            23
Densitat              23
Naixements             1
Defuncions             1
Aturats               31
Pensionistes          38
HPO acabats           18
Habitatges acabats    18
Turismes              23
Motocicletes          23
RFDH                  38
dtype: int64

In [27]:
df = df.drop(range(1975,2000))
df

,Població,Superfície,Densitat,Naixements,Defuncions,Aturats,Pensionistes,HPO acabats,Habitatges acabats,Turismes,Motocicletes,RFDH
Any,,,,,,,,,,,,
2025,97959.0,48.23,2031.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024,98649.0,48.23,2045.4,633.0,501.0,2428.8,13557.0,0.0,239.0,40087.0,12182.0,NaN
2023,97579.0,48.23,2023.2,707.0,543.0,2423.2,13199.0,61.0,150.0,40705.0,11921.0,NaN
2022,95883.0,48.23,1984.8,752.0,493.0,2353.0,12990.0,3.0,134.0,40526.0,11745.0,26038.0
2021,93937.0,48.23,1949.2,806.0,556.0,2987.4,12606.0,4.0,445.0,40318.0,11583.0,25918.0
2020,NaN,48.23,1927.8,753.0,701.0,3224.9,12269.0,2.0,399.0,40524.0,11335.0,25426.0
2019,NaN,48.23,1886.9,805.0,547.0,2567.8,12123.0,97.0,469.0,40450.0,11022.0,26201.0
2018,NaN,48.23,1879.8,808.0,526.0,2574.3,11825.0,0.0,386.0,39955.0,10752.0,24835.0
2017,NaN,48.23,1856.0,913.0,546.0,2723.3,11486.0,0.0,266.0,39374.0,10381.0,24391.0


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26 entries, 2025 to 2000
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Població            7 non-null      float64
 1   Superfície          26 non-null     float64
 2   Densitat            26 non-null     float64
 3   Naixements          25 non-null     float64
 4   Defuncions          25 non-null     float64
 5   Aturats             20 non-null     float64
 6   Pensionistes        13 non-null     float64
 7   HPO acabats         25 non-null     float64
 8   Habitatges acabats  25 non-null     float64
 9   Turismes            25 non-null     float64
 10  Motocicletes        25 non-null     float64
 11  RFDH                13 non-null     float64
dtypes: float64(12)
memory usage: 2.6 KB


In [29]:
df['Població'] = df['Població'].fillna(round(df['Densitat']* df['Superfície'], 0))
df

,Població,Superfície,Densitat,Naixements,Defuncions,Aturats,Pensionistes,HPO acabats,Habitatges acabats,Turismes,Motocicletes,RFDH
Any,,,,,,,,,,,,
2025,97959.0,48.23,2031.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024,98649.0,48.23,2045.4,633.0,501.0,2428.8,13557.0,0.0,239.0,40087.0,12182.0,NaN
2023,97579.0,48.23,2023.2,707.0,543.0,2423.2,13199.0,61.0,150.0,40705.0,11921.0,NaN
2022,95883.0,48.23,1984.8,752.0,493.0,2353.0,12990.0,3.0,134.0,40526.0,11745.0,26038.0
2021,93937.0,48.23,1949.2,806.0,556.0,2987.4,12606.0,4.0,445.0,40318.0,11583.0,25918.0
2020,92978.0,48.23,1927.8,753.0,701.0,3224.9,12269.0,2.0,399.0,40524.0,11335.0,25426.0
2019,91005.0,48.23,1886.9,805.0,547.0,2567.8,12123.0,97.0,469.0,40450.0,11022.0,26201.0
2018,90663.0,48.23,1879.8,808.0,526.0,2574.3,11825.0,0.0,386.0,39955.0,10752.0,24835.0
2017,89515.0,48.23,1856.0,913.0,546.0,2723.3,11486.0,0.0,266.0,39374.0,10381.0,24391.0


In [30]:
df.drop(2025, inplace=True)
df

,Població,Superfície,Densitat,Naixements,Defuncions,Aturats,Pensionistes,HPO acabats,Habitatges acabats,Turismes,Motocicletes,RFDH
Any,,,,,,,,,,,,
2024,98649.0,48.23,2045.4,633.0,501.0,2428.8,13557.0,0.0,239.0,40087.0,12182.0,NaN
2023,97579.0,48.23,2023.2,707.0,543.0,2423.2,13199.0,61.0,150.0,40705.0,11921.0,NaN
2022,95883.0,48.23,1984.8,752.0,493.0,2353.0,12990.0,3.0,134.0,40526.0,11745.0,26038.0
2021,93937.0,48.23,1949.2,806.0,556.0,2987.4,12606.0,4.0,445.0,40318.0,11583.0,25918.0
2020,92978.0,48.23,1927.8,753.0,701.0,3224.9,12269.0,2.0,399.0,40524.0,11335.0,25426.0
2019,91005.0,48.23,1886.9,805.0,547.0,2567.8,12123.0,97.0,469.0,40450.0,11022.0,26201.0
2018,90663.0,48.23,1879.8,808.0,526.0,2574.3,11825.0,0.0,386.0,39955.0,10752.0,24835.0
2017,89515.0,48.23,1856.0,913.0,546.0,2723.3,11486.0,0.0,266.0,39374.0,10381.0,24391.0
2016,88922.0,48.23,1843.7,939.0,509.0,3116.0,11067.0,24.0,142.0,38379.0,10075.0,24014.0


In [31]:
df.drop(range(2000,2010), inplace=True)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
Index: 15 entries, 2024 to 2010
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Població            15 non-null     float64
 1   Superfície          15 non-null     float64
 2   Densitat            15 non-null     float64
 3   Naixements          15 non-null     float64
 4   Defuncions          15 non-null     float64
 5   Aturats             15 non-null     float64
 6   Pensionistes        13 non-null     float64
 7   HPO acabats         15 non-null     float64
 8   Habitatges acabats  15 non-null     float64
 9   Turismes            15 non-null     float64
 10  Motocicletes        15 non-null     float64
 11  RFDH                13 non-null     float64
dtypes: float64(12)
memory usage: 1.5 KB


,Població,Superfície,Densitat,Naixements,Defuncions,Aturats,Pensionistes,HPO acabats,Habitatges acabats,Turismes,Motocicletes,RFDH
Any,,,,,,,,,,,,
2024,98649.0,48.23,2045.4,633.0,501.0,2428.8,13557.0,0.0,239.0,40087.0,12182.0,NaN
2023,97579.0,48.23,2023.2,707.0,543.0,2423.2,13199.0,61.0,150.0,40705.0,11921.0,NaN
2022,95883.0,48.23,1984.8,752.0,493.0,2353.0,12990.0,3.0,134.0,40526.0,11745.0,26038.0
2021,93937.0,48.23,1949.2,806.0,556.0,2987.4,12606.0,4.0,445.0,40318.0,11583.0,25918.0
2020,92978.0,48.23,1927.8,753.0,701.0,3224.9,12269.0,2.0,399.0,40524.0,11335.0,25426.0
2019,91005.0,48.23,1886.9,805.0,547.0,2567.8,12123.0,97.0,469.0,40450.0,11022.0,26201.0
2018,90663.0,48.23,1879.8,808.0,526.0,2574.3,11825.0,0.0,386.0,39955.0,10752.0,24835.0
2017,89515.0,48.23,1856.0,913.0,546.0,2723.3,11486.0,0.0,266.0,39374.0,10381.0,24391.0
2016,88922.0,48.23,1843.7,939.0,509.0,3116.0,11067.0,24.0,142.0,38379.0,10075.0,24014.0


In [32]:
df.loc[2015, 'Pensionistes'] = round((df['Pensionistes'][2016]+df['Pensionistes'][2014])/2, 0)
df.loc[2010, 'Pensionistes'] = df.loc[2011, 'Pensionistes']
df

,Població,Superfície,Densitat,Naixements,Defuncions,Aturats,Pensionistes,HPO acabats,Habitatges acabats,Turismes,Motocicletes,RFDH
Any,,,,,,,,,,,,
2024,98649.0,48.23,2045.4,633.0,501.0,2428.8,13557.0,0.0,239.0,40087.0,12182.0,NaN
2023,97579.0,48.23,2023.2,707.0,543.0,2423.2,13199.0,61.0,150.0,40705.0,11921.0,NaN
2022,95883.0,48.23,1984.8,752.0,493.0,2353.0,12990.0,3.0,134.0,40526.0,11745.0,26038.0
2021,93937.0,48.23,1949.2,806.0,556.0,2987.4,12606.0,4.0,445.0,40318.0,11583.0,25918.0
2020,92978.0,48.23,1927.8,753.0,701.0,3224.9,12269.0,2.0,399.0,40524.0,11335.0,25426.0
2019,91005.0,48.23,1886.9,805.0,547.0,2567.8,12123.0,97.0,469.0,40450.0,11022.0,26201.0
2018,90663.0,48.23,1879.8,808.0,526.0,2574.3,11825.0,0.0,386.0,39955.0,10752.0,24835.0
2017,89515.0,48.23,1856.0,913.0,546.0,2723.3,11486.0,0.0,266.0,39374.0,10381.0,24391.0
2016,88922.0,48.23,1843.7,939.0,509.0,3116.0,11067.0,24.0,142.0,38379.0,10075.0,24014.0


In [33]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 15 entries, 2024 to 2010
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Població            15 non-null     float64
 1   Superfície          15 non-null     float64
 2   Densitat            15 non-null     float64
 3   Naixements          15 non-null     float64
 4   Defuncions          15 non-null     float64
 5   Aturats             15 non-null     float64
 6   Pensionistes        15 non-null     float64
 7   HPO acabats         15 non-null     float64
 8   Habitatges acabats  15 non-null     float64
 9   Turismes            15 non-null     float64
 10  Motocicletes        15 non-null     float64
 11  RFDH                13 non-null     float64
dtypes: float64(12)
memory usage: 2.1 KB


In [34]:
df.loc[2023, 'RFDH'] = df.loc[2022, 'RFDH']
df.loc[2024, 'RFDH'] = df.loc[2022, 'RFDH']
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15 entries, 2024 to 2010
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Població            15 non-null     float64
 1   Superfície          15 non-null     float64
 2   Densitat            15 non-null     float64
 3   Naixements          15 non-null     float64
 4   Defuncions          15 non-null     float64
 5   Aturats             15 non-null     float64
 6   Pensionistes        15 non-null     float64
 7   HPO acabats         15 non-null     float64
 8   Habitatges acabats  15 non-null     float64
 9   Turismes            15 non-null     float64
 10  Motocicletes        15 non-null     float64
 11  RFDH                15 non-null     float64
dtypes: float64(12)
memory usage: 2.1 KB


In [35]:
df_stcugat = df
df_stcugat

,Població,Superfície,Densitat,Naixements,Defuncions,Aturats,Pensionistes,HPO acabats,Habitatges acabats,Turismes,Motocicletes,RFDH
Any,,,,,,,,,,,,
2024,98649.0,48.23,2045.4,633.0,501.0,2428.8,13557.0,0.0,239.0,40087.0,12182.0,26038.0
2023,97579.0,48.23,2023.2,707.0,543.0,2423.2,13199.0,61.0,150.0,40705.0,11921.0,26038.0
2022,95883.0,48.23,1984.8,752.0,493.0,2353.0,12990.0,3.0,134.0,40526.0,11745.0,26038.0
2021,93937.0,48.23,1949.2,806.0,556.0,2987.4,12606.0,4.0,445.0,40318.0,11583.0,25918.0
2020,92978.0,48.23,1927.8,753.0,701.0,3224.9,12269.0,2.0,399.0,40524.0,11335.0,25426.0
2019,91005.0,48.23,1886.9,805.0,547.0,2567.8,12123.0,97.0,469.0,40450.0,11022.0,26201.0
2018,90663.0,48.23,1879.8,808.0,526.0,2574.3,11825.0,0.0,386.0,39955.0,10752.0,24835.0
2017,89515.0,48.23,1856.0,913.0,546.0,2723.3,11486.0,0.0,266.0,39374.0,10381.0,24391.0
2016,88922.0,48.23,1843.7,939.0,509.0,3116.0,11067.0,24.0,142.0,38379.0,10075.0,24014.0
